# TetCTF 2022 Crypto

There's a nice collection of four crypto challenges here, themed after 2022. We use SageMath and pwntools to solve them.

In [1]:
%env PWNLIB_NOTERM=true
from pwn import *
from Crypto.Util.number import long_to_bytes

env: PWNLIB_NOTERM=true


In [2]:
# set local = True to test locally
# set local = False to connect to remote server
local = False

# Shares
(29 solves / 216 points)

First we have some code from the challenge that basically encodes our elements of $\mathbb{F}_{37}$ as ASCII characters.

In [3]:
ALLOWED_CHARS = string.ascii_lowercase + string.digits + "_"
P = len(ALLOWED_CHARS)
INT_TO_CHAR = {}
CHAR_TO_INT = {}
for _i, _c in enumerate(ALLOWED_CHARS):
    INT_TO_CHAR[_i] = _c
    CHAR_TO_INT[_c] = _i
    
FP = GF(P)
(ALLOWED_CHARS, FP)

('abcdefghijklmnopqrstuvwxyz0123456789_', Finite Field of size 37)

Each 16x33 matrix we get per query can be treated as three augmented parts $[A | B | c]$, where $A$ is the matrix that actually gives us useful information about our password, while $B$ only serves to add noise. It's clear that we get no information if $B$ is full rank, and we can extract $\text{nullity}(B)$ rows of information otherwise.

We do this by rearranging the matrix to $[B | c | A]$ and then reducing it to rref: $[B' | c' | A']$. Then $B' = I$ if B is full rank, and otherwise we can take the bottom $\text{nullity}(B)$ rows of the matrix. Out of laziness we only take the bottom row. $B$ is singular with probability roughly $\frac{1}{36}$ so we'd need roughly $16 \times 36 = 576$ queries to get the password. More is better, so we choose to work with 1000 queries instead. The resulting code is as follows:

In [4]:
with process(['python', 'shares.py']) if local else remote('139.162.61.222', 13371) as sh:

    runs = 1000
    sh.send(b'\n' * runs)

    efs = [Matrix(FP, [[CHAR_TO_INT[c] for c in s[16:]+s[:16]] for s in eval(sh.recvline())]).echelon_form() for _ in range(runs)]
    M = reduce(lambda a,b: a.stack(b), (ef[15,16:] for ef in efs if ef[15,15] == 0))

    ans = ''.join(INT_TO_CHAR[c] for c in M[:,1:].solve_right(M.T[0]))
    print(f'password is {ans}')
    sh.sendline(ans.encode())

    result = sh.recvline(False)
result


[x] Opening connection to 139.162.61.222 on port 13371
[x] Opening connection to 139.162.61.222 on port 13371: Trying 139.162.61.222
[+] Opening connection to 139.162.61.222 on port 13371: Done
password is 05ftz2q08bumfcj0
[*] Closed connection to 139.162.61.222 port 13371


b'TetCTF{m0r3_sh4r3s____m0r3_m0r3_m0r3_fun}'

# Shares V2
(15 solves / 804 points)

This is a slight twist to the above, where we get a full 32x33 matrix of the form $[A|b]$, but the twist is that instead of solving for $Ax=b$, we have one bit of uncertainty in each coordinate: $b_i \in \left\{(Ax)_i, (Ax)_i + 18\right\}$.

This actually paves the way for a brute-force solution, by going through all $2^{32}$ possible solutions of a single query and then testing to see which ones satisfy the next query. Since $\log_{37}(2^{32}) \approx 6.14$, we would theoretically only need about 7 more rows (so less than half of the second query) to reduce the space to a single solution.

But we can do better, trading off computation with determinism. Noting that $b-18 \equiv b+\frac{1}{2} \pmod{37}$, this means that $(Ax-b)_i \in \left\{0, \frac{1}{2}\right\}$. There's a bit of fudging here to turn $\mathbb{F}_{37}$ into $\mathbb{Z}$, and the halving means we want to take it mod $\frac{37}{2}$ instead.

This is basically an instance of the Closest Vector Problem (CVP), where we're looking to minimise the distance from $Ax$ to $b+\left(\frac{1}{4}, \ldots, \frac{1}{4}\right) \pmod{\frac{37}{2}}$. By using just two queries, we can get 64 rows of A, which allows us to construct our 65x97 matrix as follows:
$$
\left[
\begin{array}{c|c|c}
  0 & 0 & w \\
  \hline
  A & \frac{37}{2}I & b + \begin{bmatrix}\frac{1}{4} \\ \vdots \\ \frac{1}{4}\end{bmatrix}
\end{array}
\right],$$
where $w$ is a sufficiently large scalar. The idea is that in a reduced basis for this space, only the largest vector will have the $w$ component, and all the other coordinates are necessarily $\pm\frac{1}{4}$. However, LLL only gives us an approximate basis, so we are not guaranteed this. Still, it seems to work often enough, and we use $w=2$ in the following code:

In [5]:
with process(['python', 'shares_v2.py']) if local else remote('139.162.61.222', 13372) as sh:

    sh.send(b'\n' * 2)
    M = Matrix([[CHAR_TO_INT[c] for c in s] for s in eval(sh.recvline()) + eval(sh.recvline())])

    fin = block_matrix([[0, 0, 2], [M[:,:32], 37/2, M[:,32] + ones_matrix(64,1)/4]]).T.LLL()[-1]
    assert(fin[0] == 2 and all(abs(i) == 1/4 for i in fin[1:])) # this can fail sometimes, just retry

    v = vector(1/4 - n for n in fin[1:])
    ans = ''.join(INT_TO_CHAR[c] for c in matrix(FP,M).solve_right(v)[:-1])
    print(f'password is {ans}')
    sh.sendline(ans.encode())

    result = sh.recvline(False)
result

[x] Opening connection to 139.162.61.222 on port 13372
[x] Opening connection to 139.162.61.222 on port 13372: Trying 139.162.61.222
[+] Opening connection to 139.162.61.222 on port 13372: Done
password is 9xira9euc0vmiw0164oode7uy1mm6f0m
[*] Closed connection to 139.162.61.222 port 13372


b'TetCTF{but_th3_m4st3r_sh4re_1s_n0t_fun_4t_4ll}'

# Fault
(14 solves / 831 points)

The first step is to get $n$ by sending a query of `-1`. This succeeds with probability 0.5 so we do this ten times just in case.

Now, the idea is to collect enough samples of $c^{d \oplus v_i}$ so that we can pitch them against each other and obtain $c^d$. The bits of $v_i$ can be thought of as either a multiplication or division of $c^{2^n}$, depending on the $n$th bit of $d$. So if we have enough of these we can force each bit $n$ to have a net identity multiplication, without having to determine the type of each individual bit.

If we have $k$ samples, then pictorially what this means is we have the integer $k$ x 129 matrix:
$$
\left[
\begin{array}{c c}
  1 & v_1 \\
  \vdots & \vdots \\
  1 & v_k
\end{array}
\right],$$
and want to find a linear combination of the row vectors that gives $(1, 0, \ldots, 0)$.

There's quite a few ways to do this, we've chosen to augment the identity matrix on the right, reduce it to rref, and then take the first row (checking that we do indeed have the desired vector). We arbitrarily choose $k=150$ in the following code:

In [6]:
with process(['python', 'fault.py']) if local else remote('139.162.61.222', 13373) as sh:

    runs1 = 10
    runs2 = 150

    sh.send(b'-1\n' * runs1)
    n = max(eval(sh.recvline(False).decode())[1]+1 for _ in range(runs1))
    print(f'n = {n}')
    assert(n > 2) # there's a tiny chance we're unlucky enough to get n=2

    sh.send(b'c\n' * runs2)
    pairs = [eval(sh.recvline(False).decode()) for _ in range(runs2)]

    def bits129(i):
        return [int(x) for x in bin(1<<128|i)[2:]]

    ef = block_matrix([[matrix([bits129(x) for x,_ in pairs]), Integer(1)]]).echelon_form()
    assert(ef[0,0] == 1 and ef[0,1:129] == 0)

    p = product(pow(a,b,n) for (_,a),b in zip(pairs,ef[0][129:]))

    result = long_to_bytes(p)
result


[x] Opening connection to 139.162.61.222 on port 13373
[x] Opening connection to 139.162.61.222 on port 13373: Trying 139.162.61.222
[+] Opening connection to 139.162.61.222 on port 13373: Done
n = 83950661912903672095677529412851957079178700427698702143888563968405977970313444200313055503935324219421318349392842110129643106865926793454524485558808163893328130983505758824407837743577022481609882701444021264243383621406164195027230353422673582542561340656149871847992762577671076149356286848907937791371
[*] Closed connection to 139.162.61.222 port 13373


b'TetCTF{4n_unr34l1st1c_f4ult____1_th1nk}'

# Algebra
(14 solves / 831 points)

I thought this was the easiest challenge of the lot, but given the low solve rate I might have just gotten lucky.

We are given a prime $p$, some constant $C \in \mathbb{F}_p$, and an operation over $\overline{F} = \mathbb{F}_p \cup \{\infty\}$:
$$\text{op}(x_1, x_2) = \frac{x_1 + x_2 + 2Cx_1 x_2}{1 - x_1 x_2}.$$

Now, we find that the map $x \mapsto \text{op}(x, x)$ has three fixed points, given by the roots of $x(x^2+2Cx+1)$, i.e. $\left\{0, -C\pm\sqrt{C^2-1}\right\}$. Let's call the quadratic roots $u$ and $u^{-1}$ (since they must multiply to 1).

For a homomorphism to exist, the standard multiplication in $\overline{F}$ (i.e. $x \mapsto x^2$) must have also three fixed points, and indeed they are $\{0, 1, \infty\}$. So this homomorphism $f$ from $\left(\overline{F} \setminus \left\{u, u^{-1}\right\}, \text{op}\right)$ to $\left(\overline{F} \setminus \left\{0, \infty\right\}, \times\right)$ is really just a permutation of $\overline{F}$. In particular, we have the identity element $f(0) = 1$ and the removed elements $f(u) = 0$ and $f(u^{-1}) = \infty$ (wlog).

If we think of $\overline{F}$ like the extended complex numbers $\overline{\mathbb{C}}$, then it seems reasonable that $f$ is going to be a rational function. In particular, since we only have three known mappings, we hypothesise that $f$ might be a [Möbius transformation](https://en.wikipedia.org/wiki/M%C3%B6bius_transformation). Indeed, if we define
$$f(x) = \frac{1 - u^{-1}x}{1-ux},$$
then it is a very easy check that $f(0)=1$, $f(u)=0$, $f(u^{-1})=\infty$, and $f(\text{op}(x_1,x_2))=f(x_1)f(x_2)$ as required.

In [7]:
with process(['python', 'algebra.py']) if local else remote('139.162.61.222', 13374) as sh:
    F = GF(50824208494214622675210983238467313009841434758617398532295301998201478298245257311594403096942992643947506323356996857413985105233960391416730079425326309)
    C = F(803799120267736039902689148809657862377959420031713529926996228010552678684828445053154435325462622566051992510975853540073683867248578880146673607388918)

    u = -C + sqrt(C * C - 1)
    print(f'u = {u}')
    def f(x):
        return (1 - x / u) / (1 - x * u)

    a = F(sh.recvlineS())
    b = F(sh.recvlineS())
    c = F(sh.recvlineS())
    sh.sendline(str(f(a)).encode())
    sh.sendline(str(f(b)).encode())
    sh.sendline(str(f(c)).encode())

    result = sh.recvline(False)
result

[x] Opening connection to 139.162.61.222 on port 13374
[x] Opening connection to 139.162.61.222 on port 13374: Trying 139.162.61.222
[+] Opening connection to 139.162.61.222 on port 13374: Done
u = 20421182952725129070409994555123927347778135528970946295972034299120598615469949023797536958799878976201068192866200304310169610609018419272022187833495020
[*] Closed connection to 139.162.61.222 port 13374


b'TetCTF{1_just_l0v3_th3_un1t_c1rcl3_s0_much}'